In [1]:
print("ok")

ok


In [2]:
import unstructured
from importlib.metadata import version

In [3]:
print(version("unstructured"))

0.18.15


In [4]:
import os
from dotenv import load_dotenv

if os.getenv('AZURE_OPENAI_API_KEY'):
    print("AzureOpenAI API key loaded successfully")
else:
    print("API KEY not found")

AzureOpenAI API key loaded successfully


In [5]:
from unstructured.partition.auto import partition
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.html import partition_html
from unstructured.partition.md import partition_md
from unstructured.partition.docx import partition_docx
from unstructured.partition.xlsx import partition_xlsx
from unstructured.partition.pptx import partition_pptx
from unstructured.partition.image import partition_image

from unstructured.chunking.title import chunk_by_title
from unstructured.chunking.basic import chunk_elements

from unstructured.documents.elements import (
    Title,
    NarrativeText,
    Table,
    ListItem,
    Image,
    Header,
    Footer,
    Text,
    ElementMetadata
)

import json
from pathlib import Path

f:\sourab\data_parsing\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
html_path = 'sample_documents\sample.html'
elements = partition(html_path)

for i, element in enumerate(elements,1):
    element_type = type(element).__name__
    text_preview = element.text[:100]+"..." if len(element.text)> 100 else element.text
    print(f"[{i}] {element_type}: {text_preview}")
    print("-"*60)


libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


[1] Title: 1. Overview
------------------------------------------------------------
[2] NarrativeText: Document parsing is the process of analyzing and extracting structured information from various docu...
------------------------------------------------------------
[3] Title: 1.1 Key Benefits
------------------------------------------------------------
[4] ListItem: Automated data extraction
------------------------------------------------------------
[5] ListItem: Structured content analysis
------------------------------------------------------------
[6] ListItem: Integration with AI/ML pipelines
------------------------------------------------------------
[7] ListItem: Support for multiple formats
------------------------------------------------------------
[8] Title: 2. Core Features
------------------------------------------------------------
[9] NarrativeText: Modern document parsers offer a variety of features:
------------------------------------------------------------
[10] 

In [7]:
title_element = [e for e in elements if isinstance(e, Title)]

if title_element:
    first_title = title_element[0]

    print("Element Text: ")
    print(first_title.text)
    print('-'*60)

    print("Element Metadata: ")
    meta_data = first_title.metadata

    print(meta_data)
    meta_data_dict = meta_data.to_dict()
    for key, value in meta_data_dict.items():
        if value is not None:
            print(f"   {key}: {value}")

Element Text: 
1. Overview
------------------------------------------------------------
Element Metadata: 
   category_depth: 1
   last_modified: 2025-12-30T05:14:11
   languages: ['eng']
   file_directory: sample_documents
   filename: sample.html
   filetype: text/html


In [8]:
def analyze_elements(elements):
    type_count = {}
    for element in elements:
        element_type = type(element).__name__
        type_count[element_type] = type_count.get(element_type,0) + 1
    print("Element Distribution: ")
    print("-"*30)
    for elem_type, count in sorted(type_count.items()):
        print(f"  {elem_type}: {count}")
    return type_count

In [9]:
type_count = analyze_elements(elements)

Element Distribution: 
------------------------------
  ListItem: 8
  NarrativeText: 6
  Table: 1
  Title: 5


In [10]:
narrative_texts = [e for e in elements if isinstance(e, NarrativeText)]
print(f"Found {len(narrative_texts)} Narrative elements")
for i, text in enumerate(narrative_texts[:5],1):
    print(f"{i}. {text.text}\n")

Found 6 Narrative elements
1. Document parsing is the process of analyzing and extracting structured information from various document formats. This includes PDFs, Word documents, HTML pages, and more.

2. Modern document parsers offer a variety of features:

3. Retrieval-Augmented Generation (RAG) systems benefit significantly from proper document parsing:

4. "Effective document parsing is the foundation of any successful RAG implementation."

5. Here's a simple example of using a document parser:



In [11]:
table_elements = [e for e in elements if isinstance(e,Table)]
print(f"Found {len(table_elements)} Table elements: \n")
for i, table in enumerate(table_elements):
    print(f"Table {i+1}: \n")
    print(table.text)
    print()

    if hasattr(table.metadata, 'text_as_html') and table.metadata.text_as_html:
        print("HTML representation available in metadata.text_as_html")
    print("-"*60)

Found 1 Table elements: 

Table 1: 

Feature Description Use Case OCR Support Optical Character Recognition for scanned documents Scanned PDFs, Images Table Extraction Structured table data extraction Financial reports, Data tables Layout Analysis Understanding document structure Academic papers, Legal documents Image Processing Extract and classify images Technical manuals, Presentations

HTML representation available in metadata.text_as_html
------------------------------------------------------------


### **Partitioning Strategies**

In [12]:
import urllib.request
import os

os.makedirs("sample_documents", exist_ok=True)
pdf_url = "https://arxiv.org/pdf/2408.09869"
pdf_path = "sample_documents\docling_paper.pdf"

if not os.path.exists(pdf_path):
    print(f"Downloading PDF from {pdf_url}")
    urllib.request.urlretrieve(pdf_url)
    print(f"Downloaded to {pdf_path}")
else:
    print(f"PDF exist at: {pdf_path}")

PDF exist at: sample_documents\docling_paper.pdf


In [13]:
import time
print("Strategy: AUTO (default)")
print("="*60)

start_time = time.time()
element_auto = partition(
    filename=pdf_path,
    strategy="auto"
)
elapsed_time = time.time()-start_time

print(f"Time taken: {elapsed_time:.2f} seconds")
print(f"Elements extracted: {len(element_auto)}")

analyze_elements(element_auto)

Strategy: AUTO (default)
Time taken: 2.05 seconds
Elements extracted: 276
Element Distribution: 
------------------------------
  Footer: 10
  ListItem: 4
  NarrativeText: 75
  Text: 86
  Title: 101


{'Text': 86, 'Title': 101, 'NarrativeText': 75, 'Footer': 10, 'ListItem': 4}

In [14]:
print("Strategy: FAST (default)")
print("="*60)

start_time = time.time()
element_fast = partition(
    filename=pdf_path,
    strategy="fast"
)
elapsed_time = time.time()-start_time

print(f"Time taken: {elapsed_time:.2f} seconds")
print(f"Elements extracted: {len(element_auto)}")

analyze_elements(element_auto)

Strategy: FAST (default)
Time taken: 1.76 seconds
Elements extracted: 276
Element Distribution: 
------------------------------
  Footer: 10
  ListItem: 4
  NarrativeText: 75
  Text: 86
  Title: 101


{'Text': 86, 'Title': 101, 'NarrativeText': 75, 'Footer': 10, 'ListItem': 4}

In [15]:
print("Strategy: HI_RES")
print("="*60)
print("Note: hi_res strategy uses ML models and may take longer")
print()

try:
    start_time = time.time()
    element_auto = partition(
        filename=pdf_path,
        strategy="hi_res"
    )
    elapsed_time = time.time()-start_time

    print(f"Time taken: {elapsed_time:.2f} seconds")
    print(f"Elements extracted: {len(element_auto)}")

    analyze_elements(element_auto)
except Exception as e:
    print(f"Error: {str(e)}")

Strategy: HI_RES
Note: hi_res strategy uses ML models and may take longer

Time taken: 56.05 seconds
Elements extracted: 341
Element Distribution: 
------------------------------
  FigureCaption: 2
  Footer: 2
  Header: 4
  Image: 24
  ListItem: 20
  NarrativeText: 58
  Table: 4
  Text: 202
  Title: 25


In [16]:
def show_first_elements(elements, n=5, strategy_name=""):
    print(f"\n{'='*60}")
    print(f"First {n} elements from {strategy_name} strategy:")
    print("="*60)

    for i, elem in enumerate(elements[:n]):
        elem_tp = type(elem).__name__
        text = elem.text[:150]+"..." if len(elem.text)>150 else elem.text
        print(f"[{i+1}] {elem_tp}")
        print(f"   {text}")

show_first_elements(element_auto, n=5, strategy_name="AUTO")
show_first_elements(element_fast, n=5, strategy_name="AUTO")


First 5 elements from AUTO strategy:
[1] Text
   4
[2] Text
   2024
[3] Text
   2
[4] Text
   0
[5] Text
   2

First 5 elements from AUTO strategy:
[1] Text
   4 2 0 2 c e D 9
[2] Title
   ] L C . s c [
[3] Text
   5 v 9 6 8 9 0 . 8 0 4 2 : v i X r a
[4] Title
   Docling Technical Report
[5] Title
   Version 1.0


In [17]:
from unstructured.partition.pdf import partition_pdf

pdf_element = partition_pdf(
    filename=pdf_path,
    strategy='fast'
)
print(f"PDF Elements: {len(pdf_element)}")
print(f"\nFirst 3 elements: ")
for elem in pdf_element[:3]:
    print(f"    - {type(elem).__name__}: {elem.text[:80]}...")
    

PDF Elements: 276

First 3 elements: 
    - Text: 4 2 0 2 c e D 9...
    - Title: ] L C . s c [...
    - Text: 5 v 9 6 8 9 0 . 8 0 4 2 : v i X r a...


In [18]:
pdf_elements = partition_pdf(
    filename=pdf_path,
    strategy='fast',
    include_page_breaks=True
)
tables = [e for e in pdf_elements if isinstance(e, Table)]
print(f"Found {len(tables)} tables in the PDF")

if tables:
    print(f"First table content: ")
    print(tables[0].text[:500])

Found 0 tables in the PDF


In [19]:
pages = {}
for elm in pdf_elements:
    page_num = elm.metadata.page_number if elm.metadata.page_number else 0
    if page_num not in pages:
        pages[page_num] = []
    pages[page_num].append(elm)
print(f"Documents has {len(pages)} pages")
for page_num in sorted(pages.keys())[:3]:
    print(f"\nPage {page_num}: {len(pages[page_num])} elements")

Documents has 10 pages

Page 0: 9 elements

Page 1: 15 elements

Page 2: 19 elements


In [20]:
html_elements = partition_html(
    filename="sample_documents\sample.html"
)
print(f"HTML Elements: {len(html_elements)}")
analyze_elements(html_elements)

HTML Elements: 20
Element Distribution: 
------------------------------
  ListItem: 8
  NarrativeText: 6
  Table: 1
  Title: 5


{'Title': 5, 'NarrativeText': 6, 'ListItem': 8, 'Table': 1}

In [21]:
try:
    url_element = partition_html(
        url="https://docling-project.github.io/docling/"
    )
    print(f"Element fetch from URL: {len(url_element)}")
    for elm in url_element[:5]:
        print(f"   - {type(elm).__name__}: {elm.text[:60]}...")
except Exception as e:
    print(f"Could not fetch URL :{e}")

Element fetch from URL: 54
   - Title: Documentation...
   - Image: Docling...
   - Image: DS4SD%2Fdocling | Trendshift...
   - Image: arXiv...
   - Image: PyPI version...


In [22]:
html_content = """
<html>
<body>
    <h1>Sample Document</h1>
    <p>This is a paragraph with some <strong>bold text</strong>.</p>
    <ul>
        <li>Item 1</li>
        <li>Item 2</li>
        <li>Item 3</li>
    </ul>
    <table>
        <tr><th>Name</th><th>Value</th></tr>
        <tr><td>A</td><td>100</td></tr>
        <tr><td>B</td><td>200</td></tr>
    </table>
</body>
</html>
"""
elements_from_string = partition_html(
    text=html_content
)
print(f"Elements from string: {len(elements_from_string)}")
for elem in elements_from_string:
    print(f"  - {type(elem).__name__}: {elem.text}")

Elements from string: 6
  - Title: Sample Document
  - NarrativeText: This is a paragraph with some bold text.
  - ListItem: Item 1
  - ListItem: Item 2
  - ListItem: Item 3
  - Table: Name Value A 100 B 200


In [23]:
md_elements = partition_md(filename="sample_documents\sample.md")

print(f"Markdown Elements: {len(md_elements)}")
analyze_elements(md_elements)

Markdown Elements: 44
Element Distribution: 
------------------------------
  ListItem: 15
  NarrativeText: 12
  Table: 2
  Text: 1
  Title: 14


{'Title': 14, 'NarrativeText': 12, 'ListItem': 15, 'Table': 2, 'Text': 1}

In [24]:
print("Markdown Structure:")
print("="*60)

for elem in md_elements[:15]:
    element_type = type(elem).__name__
    text = elem.text[:80]+"..." if len(elem.text)>80 else elem.text
    print(f"{element_type:15} | {text}")

Markdown Structure:
Title           | Document Parsing Best Practices
NarrativeText   | A comprehensive guide to document parsing for RAG systems.
Title           | Table of Contents
ListItem        | Introduction
ListItem        | Supported Formats
ListItem        | Parsing Strategies
ListItem        | Integration Guide
Title           | Introduction
NarrativeText   | Document parsing is a critical component in modern AI applications. It enables t...
ListItem        | Build searchable knowledge bases
ListItem        | Create training datasets for machine learning
ListItem        | Enable semantic search and retrieval
ListItem        | Power question-answering systems
NarrativeText   | Note: The quality of document parsing directly impacts the performance of downst...
Title           | Supported Formats


In [25]:
md_string = """
# Main Title

This is an introduction paragraph.

## Section 1

Here's some content with:
- Bullet point 1
- Bullet point 2

## Section 2

```python
def hello():
    print("Hello, World!")
```

| Column A | Column B |
|----------|----------|
| Value 1  | Value 2  |
"""

md_string_elements = partition_md(text=md_string)

print(f"Elements from markdown string: {len(md_string_elements)}")
for elem in md_string_elements:
    print(f"  - {type(elem).__name__}: {elem.text[:60]}..." if len(elem.text) > 60 else f"  - {type(elem).__name__}: {elem.text}")

Elements from markdown string: 7
  - Title: Main Title
  - NarrativeText: This is an introduction paragraph.
  - Title: Section 1
  - NarrativeText: Here's some content with: - Bullet point 1 - Bullet point 2
  - Title: Section 2
  - Text: def hello():
    print("Hello, World!")
  - Table: Column A Column B Value 1 Value 2


In [26]:
try:
    from docx import Document as DocxDocument
    from docx.shared import Inches
    
    # Create a new document
    doc = DocxDocument()
    
    # Add content
    doc.add_heading('Sample Word Document', 0)
    doc.add_paragraph('This is a sample Word document created for testing Unstructured.')
    
    doc.add_heading('Section 1: Introduction', level=1)
    doc.add_paragraph('Unstructured is a powerful library for document parsing.')
    
    doc.add_heading('Section 2: Features', level=1)
    doc.add_paragraph('Key features include:')
    
    # Add a bulleted list
    doc.add_paragraph('Multiple file format support', style='List Bullet')
    doc.add_paragraph('OCR capabilities', style='List Bullet')
    doc.add_paragraph('Table extraction', style='List Bullet')
    
    # Add a simple table
    table = doc.add_table(rows=3, cols=2)
    table.style = 'Table Grid'
    cells = table.rows[0].cells
    cells[0].text = 'Feature'
    cells[1].text = 'Status'
    cells = table.rows[1].cells
    cells[0].text = 'PDF Support'
    cells[1].text = 'Available'
    cells = table.rows[2].cells
    cells[0].text = 'OCR'
    cells[1].text = 'Available'
    
    # Save the document
    docx_path = 'sample_documents/sample1.docx'
    doc.save(docx_path)
    print(f"Created sample DOCX at: {docx_path}")
    
except ImportError:
    print("python-docx not installed. Install with: pip install python-docx")
    docx_path = None

Created sample DOCX at: sample_documents/sample1.docx


In [27]:
if docx_path and os.path.exists(docx_path):
    docx_elements = partition_docx(filename=docx_path)
    
    print(f"DOCX Elements: {len(docx_elements)}")
    analyze_elements(docx_elements)
    
    print("\nContent:")
    for elem in docx_elements:
        print(f"  - {type(elem).__name__}: {elem.text}")
else:
    print("No DOCX file available for demo")

DOCX Elements: 10
Element Distribution: 
------------------------------
  ListItem: 3
  NarrativeText: 3
  Table: 1
  Title: 3

Content:
  - Title: Sample Word Document
  - NarrativeText: This is a sample Word document created for testing Unstructured.
  - Title: Section 1: Introduction
  - NarrativeText: Unstructured is a powerful library for document parsing.
  - Title: Section 2: Features
  - NarrativeText: Key features include:
  - ListItem: Multiple file format support
  - ListItem: OCR capabilities
  - ListItem: Table extraction
  - Table: Feature Status PDF Support Available OCR Available


In [28]:
try:
    import openpyxl
    
    # Create a new workbook
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Data"
    
    # Add headers
    ws['A1'] = 'Product'
    ws['B1'] = 'Category'
    ws['C1'] = 'Price'
    ws['D1'] = 'Quantity'
    
    # Add data
    data = [
        ('Laptop', 'Electronics', 999.99, 50),
        ('Phone', 'Electronics', 599.99, 100),
        ('Desk', 'Furniture', 299.99, 25),
        ('Chair', 'Furniture', 149.99, 75),
        ('Monitor', 'Electronics', 349.99, 40),
    ]
    
    for row_idx, row_data in enumerate(data, start=2):
        for col_idx, value in enumerate(row_data, start=1):
            ws.cell(row=row_idx, column=col_idx, value=value)
    
    # Save the workbook
    xlsx_path = 'sample_documents/sample1.xlsx'
    wb.save(xlsx_path)
    print(f"Created sample XLSX at: {xlsx_path}")
    
except ImportError:
    print("openpyxl not installed. Install with: pip install openpyxl")
    xlsx_path = None

Created sample XLSX at: sample_documents/sample1.xlsx


In [29]:
xlsx_path = "sample_documents/sample.xlsx"
if xlsx_path and os.path.exists(xlsx_path):
    xlsx_elements = partition_xlsx(filename=xlsx_path)
    
    print(f"XLSX Elements: {len(xlsx_elements)}")
    analyze_elements(xlsx_elements)
    
    print("\nContent:")
    for elem in xlsx_elements:
        text = elem.text[:200] + "..." if len(elem.text) > 200 else elem.text
        print(f"  - {type(elem).__name__}: {text}")
else:
    print("No XLSX file available for demo")

XLSX Elements: 11
Element Distribution: 
------------------------------
  NarrativeText: 4
  Table: 3
  Text: 2
  Title: 2

Content:
  - Table: Dates Modules 2 November 2025 9 November 2025 16 November 2025 23 November 2025 30 November 2025 7 December 2025 Module 4: Document Parsers + Module 5 (Part 1): LlamaIndex Fundamentals 14 December 202...
  - Text: 12 April 2026
  - Title: Black Friday 2025: AI Services & Tools Discount Guide
  - Table: by Seulki Kang https://www.linkedin.com/in/seulki-kang/
  - Title: Crawled by Genspark AI Sheet
  - NarrativeText: Want to try Genspark yourself?
  - NarrativeText: 🔗 Get 1,000 free credits: https://www.genspark.ai/invite_member?invite_code=NTQxMmI5M2ZMOTRhMUwyODg2TDhjZmZMNzE3ODRhYmI0Y2Jl
  - NarrativeText: 🔗 10% discount code: https://www.genspark.ai/?via=seulki875 (US) / https://www.genspark.ai/?via=seulki (Other Countries)
  - Table: Name Description URL Benefits Monthly Plan Annual Plan Notes ⭐ Popular AI Services (Discount Info) Perplexity P

In [30]:
try:
    from pptx import Presentation
    from pptx.util import Inches, Pt
    
    # Create a presentation
    prs = Presentation()
    
    # Add title slide
    title_slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(title_slide_layout)
    title = slide.shapes.title
    subtitle = slide.placeholders[1]
    title.text = "Unstructured Demo"
    subtitle.text = "Document Parsing for RAG"
    
    # Add content slide
    bullet_slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(bullet_slide_layout)
    shapes = slide.shapes
    title_shape = shapes.title
    body_shape = shapes.placeholders[1]
    title_shape.text = "Key Features"
    tf = body_shape.text_frame
    tf.text = "Multiple file formats supported"
    p = tf.add_paragraph()
    p.text = "OCR for scanned documents"
    p.level = 0
    p = tf.add_paragraph()
    p.text = "Table extraction"
    p.level = 0
    
    # Save the presentation
    pptx_path = 'sample_documents/sample1.pptx'
    prs.save(pptx_path)
    print(f"Created sample PPTX at: {pptx_path}")
    
except ImportError:
    print("python-pptx not installed. Install with: pip install python-pptx")
    pptx_path = None

Created sample PPTX at: sample_documents/sample1.pptx


In [32]:
from unstructured.partition.pptx import partition_pptx

pptx_path = "sample_documents/sample1.pptx"
if pptx_path and os.path.exists(pptx_path):
    pptx_elements = partition_pptx(filename=pptx_path)
    
    print(f"PPTX Elements: {len(pptx_elements)}")
    analyze_elements(pptx_elements)
    
    print("\nContent:")
    for elem in pptx_elements:
        print(f"  - {type(elem).__name__}: {elem.text}")
else:
    print("No PPTX file available for demo")

PPTX Elements: 7
Element Distribution: 
------------------------------
  NarrativeText: 1
  PageBreak: 1
  Title: 5

Content:
  - Title: Unstructured Demo
  - Title: Document Parsing for RAG
  - PageBreak: 
  - Title: Key Features
  - NarrativeText: Multiple file formats supported
  - Title: OCR for scanned documents
  - Title: Table extraction


In [33]:
try:
    from PIL import Image, ImageDraw, ImageFont
    
    # Create a white image
    img = Image.new('RGB', (800, 400), color='white')
    draw = ImageDraw.Draw(img)
    
    # Try to use a basic font, fall back to default if not available
    try:
        font_title = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 36)
        font_text = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
    except:
        font_title = ImageFont.load_default()
        font_text = ImageFont.load_default()
    
    # Add text to the image
    draw.text((50, 30), "Document Parsing Demo", fill='black', font=font_title)
    draw.text((50, 100), "This is sample text for OCR testing.", fill='black', font=font_text)
    draw.text((50, 150), "Unstructured can extract text from images.", fill='black', font=font_text)
    draw.text((50, 200), "Features:", fill='black', font=font_text)
    draw.text((70, 250), "• Multiple language support", fill='black', font=font_text)
    draw.text((70, 300), "• Various OCR backends", fill='black', font=font_text)
    draw.text((70, 350), "• High accuracy extraction", fill='black', font=font_text)
    
    # Save the image
    image_path = 'sample_documents/sample_text_image.png'
    img.save(image_path)
    print(f"Created sample image at: {image_path}")
    
except ImportError:
    print("Pillow not installed. Install with: pip install Pillow")
    image_path = None

Created sample image at: sample_documents/sample_text_image.png


In [34]:

from unstructured.partition.image import partition_image

if image_path and os.path.exists(image_path):
    try:
        # Partition the image using OCR
        image_elements = partition_image(
            filename=image_path,
            strategy="auto",  # Will use OCR automatically
        )
        
        print(f"Image Elements: {len(image_elements)}")
        analyze_elements(image_elements)
        
        print("\nExtracted Text:")
        for elem in image_elements:
            print(f"  - {type(elem).__name__}: {elem.text}")
            
    except Exception as e:
        print(f"OCR error: {e}")
        print("\nOCR requires additional dependencies.")
        print("Install with: pip install 'unstructured[local-inference]'")
        print("Also ensure Tesseract OCR is installed on your system.")
else:
    print("No image file available for demo")

Image Elements: 7
Element Distribution: 
------------------------------
  Footer: 1
  Header: 1
  NarrativeText: 5

Extracted Text:
  - Header: Document Parsing Demo
  - NarrativeText: This issample text for OCR testing.
  - NarrativeText: Unstructured can extract text from images.
  - NarrativeText: Features
  - NarrativeText: 8 Multiple language support
  - NarrativeText: 8 VariousOCR backends
  - Footer: 8 High accuracy extraction


### **Element Types Deep Dive**

In [36]:
titles = [e for e in md_elements if isinstance(e,Title)]
print("Title Element Analysis: ")
print("="*60)
print(f"Total title found: {len(titles)}")
print()

for i, title in enumerate(titles):
    print(f"Title {i+1}")
    print(f"   Text: {title.text}")
    print(f"   ID: {title.id}")

    if hasattr(title.metadata, 'category_depth'):
        print(f"   Category Depth: {title.metadata.category_depth}")
    print()

Title Element Analysis: 
Total title found: 14

Title 1
   Text: Document Parsing Best Practices
   ID: 9ea1b6ba00245b0da1c77098a9e130d5
   Category Depth: 0

Title 2
   Text: Table of Contents
   ID: bf0aeb6089cc5feb43bf6f5734f3d849
   Category Depth: 1

Title 3
   Text: Introduction
   ID: be39b6913500d3c48b592988ca400449
   Category Depth: 1

Title 4
   Text: Supported Formats
   ID: bfe92481356969a08531b7d9e04d44ac
   Category Depth: 1

Title 5
   Text: Primary Formats
   ID: e88f38a1f4f2fab33f199c2dd9224d98
   Category Depth: 2

Title 6
   Text: Additional Formats
   ID: f6dba65dcea180547f8a68ae26c5dba6
   Category Depth: 2

Title 7
   Text: Parsing Strategies
   ID: 8a928a0b60821feb87644c6fc21c9f70
   Category Depth: 1

Title 8
   Text: 1. Standard Pipeline
   ID: 314dbd3bfb1c1d5652cc0bdb9628d1b3
   Category Depth: 2

Title 9
   Text: 2. VLM Pipeline
   ID: 5f47990bee5c80bd8d01c49846ead923
   Category Depth: 2

Title 10
   Text: 3. OCR Configuration
   ID: 6215c31987d2dc9dd960df1

In [41]:
narrative_text = [e for e in md_elements if isinstance(e,NarrativeText)]
print("Title Element Analysis: ")
print("="*60)
print(f"Total NarrativeText found: {len(narrative_text)}")
print()

total_chars = sum(len(n.text) for n in narrative_text)
avg_text = total_chars / len(narrative_text) if narrative_text else 0

print(f"Total characters: {total_chars}")
print(f"Average charactors per elements: {avg_text:.1f}")
print()

print("Sample narrative texts: ")
for i, n in enumerate(narrative_text[:3]):
    preview = n.text[:100]+"..." if len(n.text)> 100 else n.text
    print(f"   - {preview}")

Title Element Analysis: 
Total NarrativeText found: 12

Total characters: 1227
Average charactors per elements: 102.2

Sample narrative texts: 
   - A comprehensive guide to document parsing for RAG systems.
   - Document parsing is a critical component in modern AI applications. It enables the extraction of str...
   - Note: The quality of document parsing directly impacts the performance of downstream AI applications...


In [42]:
tables = [e for e in html_elements if isinstance(e, Table)]

print("Table Elements Analysis")
print("="*60)
print(f"Total tables: {len(tables)}")
print()

for i, table in enumerate(tables):
    print(f"Table {i+1}:")
    print(f"  Content: {table.text[:300]}..." if len(table.text) > 300 else f"  Content: {table.text}")
    
    # Check for HTML representation
    if hasattr(table.metadata, 'text_as_html') and table.metadata.text_as_html:
        print(f"  HTML available: Yes")
    print()

Table Elements Analysis
Total tables: 1

Table 1:
  Content: Feature Description Use Case OCR Support Optical Character Recognition for scanned documents Scanned PDFs, Images Table Extraction Structured table data extraction Financial reports, Data tables Layout Analysis Understanding document structure Academic papers, Legal documents Image Processing Extrac...
  HTML available: Yes



In [46]:
list_items = [e for e in html_elements if isinstance(e, ListItem)]

print("ListItem Elements Analysis")
print("="*60)
print(f"Total list items: {len(list_items)}")
print()

print("List items found:")
for item in list_items:
    print(f"  • {item.text}")
    #print(type(item).__name__)

ListItem Elements Analysis
Total list items: 8

List items found:
  • Automated data extraction
  • Structured content analysis
  • Integration with AI/ML pipelines
  • Support for multiple formats
  • Knowledge Base Creation: Convert documents into searchable chunks
  • Semantic Search: Enable meaning-based document retrieval
  • Question Answering: Provide accurate answers from document context
  • Document Summarization: Generate concise summaries


In [48]:
element_dict = html_elements[0].to_dict()
print("Element as Dictionary: ")
print(json.dumps(element_dict, indent=2))

Element as Dictionary: 
{
  "type": "Title",
  "element_id": "2a000fb58dc445e313c5dffe0ef3a614",
  "text": "1. Overview",
  "metadata": {
    "category_depth": 1,
    "last_modified": "2025-12-30T05:14:11",
    "languages": [
      "eng"
    ],
    "file_directory": "sample_documents",
    "filename": "sample.html",
    "filetype": "text/html"
  }
}


In [50]:
from unstructured.staging.base import elements_to_json

json_output = elements_to_json(html_elements[:5])
print("First 5 elements as JSON")
print(json_output[:1000]+"..." if len(json_output)>1000 else json_output)

First 5 elements as JSON
[
    {
        "element_id": "2a000fb58dc445e313c5dffe0ef3a614",
        "metadata": {
            "category_depth": 1,
            "file_directory": "sample_documents",
            "filename": "sample.html",
            "filetype": "text/html",
            "languages": [
                "eng"
            ],
            "last_modified": "2025-12-30T05:14:11"
        },
        "text": "1. Overview",
        "type": "Title"
    },
    {
        "element_id": "0111c3838e668907531ba2247a8d97b1",
        "metadata": {
            "file_directory": "sample_documents",
            "filename": "sample.html",
            "filetype": "text/html",
            "languages": [
                "eng"
            ],
            "last_modified": "2025-12-30T05:14:11",
            "parent_id": "2a000fb58dc445e313c5dffe0ef3a614"
        },
        "text": "Document parsing is the process of analyzing and extracting structured information from various document formats. This inclu

### **Working with Metadata**

In [52]:
sample_element = pdf_element[0]
print("Element Metadata Field: ")
print("="*60)

metadata_dict = sample_element.metadata.to_dict()
for k,v in metadata_dict.items():
    if v is not None:
        print(f"{k:15}: {v}")

Element Metadata Field: 
coordinates    : {'points': ((16.34, 216.16000000000008), (16.34, 308.36), (36.34, 308.36), (36.34, 216.16000000000008)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}
file_directory : sample_documents
filename       : docling_paper.pdf
last_modified  : 2025-12-30T05:14:11
page_number    : 1
languages      : ['eng']
filetype       : application/pdf


In [53]:
print("Common Metadata Fields:")
print("="*60)

metadata_info = {
    "filename": "Source file name - useful for citations",
    "file_directory": "Directory path of the source file",
    "filetype": "MIME type of the source document",
    "page_number": "Page number in the document",
    "coordinates": "Bounding box coordinates (for layout analysis)",
    "text_as_html": "HTML representation (for tables)",
    "category_depth": "Heading level (for titles)",
    "languages": "Detected languages in the text",
    "emphasized_text_contents": "Bold/italic text",
    "link_urls": "URLs found in the element",
}
for field, description in metadata_info.items():
    print(f"\n{field}")
    print(f"   {description}")

Common Metadata Fields:

filename
   Source file name - useful for citations

file_directory
   Directory path of the source file

filetype
   MIME type of the source document

page_number
   Page number in the document

coordinates
   Bounding box coordinates (for layout analysis)

text_as_html
   HTML representation (for tables)

category_depth
   Heading level (for titles)

languages
   Detected languages in the text

emphasized_text_contents
   Bold/italic text

link_urls
   URLs found in the element


In [54]:
page_1_elements = [
    e for e in pdf_elements if e.metadata.page_number==1
]
print(f"Element on page 1: {len(page_1_elements)}")
for elem in page_1_elements[:5]:
    text = elem.text[:80]+'...' if len(elem.text)>80 else elem.text
    print(f"   - {type(elem).__name__}: {text}")

Element on page 1: 15
   - Text: 4 2 0 2 c e D 9
   - Title: ] L C . s c [
   - Text: 5 v 9 6 8 9 0 . 8 0 4 2 : v i X r a
   - Title: Docling Technical Report
   - Title: Version 1.0


In [55]:
from datetime import datetime
processed_elements = []

for elem in html_elements[:3]:
    elem.metadata.detection_dup = "unstructured-notebook-demo"
    processed_elements.append(elem)
print("Elements with custom metadata:")
for elem in processed_elements:
    print(f"   - {elem.text[:50]}")
    print(f"   - detection_dup: {elem.metadata.detection_dup}")

Elements with custom metadata:
   - 1. Overview
   - detection_dup: unstructured-notebook-demo
   - Document parsing is the process of analyzing and e
   - detection_dup: unstructured-notebook-demo
   - 1.1 Key Benefits
   - detection_dup: unstructured-notebook-demo


In [56]:
ocr_options = {
    "tesseract": {
        "description": "Google's Tesseract OCR - widely used, good accuracy",
        "installation": "apt-get install tesseract-ocr",
        "languages": "100+ languages supported",
    },
    "paddle": {
        "description": "PaddleOCR - excellent for multi-language documents",
        "installation": "pip install paddlepaddle paddleocr",
        "languages": "80+ languages, strong CJK support",
    },
}

print("Available OCR Backends:")
print("="*60)

for backend, info in ocr_options.items():
    print(f"\n{backend.upper()}:")
    for key, value in info.items():
        print(f"  {key}: {value}")

Available OCR Backends:

TESSERACT:
  description: Google's Tesseract OCR - widely used, good accuracy
  installation: apt-get install tesseract-ocr
  languages: 100+ languages supported

PADDLE:
  description: PaddleOCR - excellent for multi-language documents
  installation: pip install paddlepaddle paddleocr
  languages: 80+ languages, strong CJK support


In [58]:
ocr_config = {
    "strategy":"hi_res",
    "ocr_languages":"eng",
    "hi_res_model_name":"yolox"
}

print("OCR Configuaration Example:")
print(json.dumps(ocr_config, indent=2))

image_path = "sample_documents\sample_text_image.png"

if image_path and os.path.exists(image_path):
    try:
        ocr_elements = partition_image(
            filename=image_path,
            strategy="hi_res",
            ocr_languages="eng"
        )
        print(f"\nOCR extracted {len(ocr_elements)} elements")
        for elem in ocr_elements:
            print(f"{type(elem).__name__}: {elem.text}")
    except Exception as e:
        print(f"\nOCR not available: {e}")

The ocr_languages kwarg will be deprecated in a future version of unstructured. Please use languages instead.


Only one of languages and ocr_languages should be specified. languages is preferred. ocr_languages is marked for deprecation.


OCR Configuaration Example:
{
  "strategy": "hi_res",
  "ocr_languages": "eng",
  "hi_res_model_name": "yolox"
}

OCR extracted 7 elements
Header: Document Parsing Demo
NarrativeText: This issample text for OCR testing.
NarrativeText: Unstructured can extract text from images.
NarrativeText: Features
NarrativeText: 8 Multiple language support
NarrativeText: 8 VariousOCR backends
Footer: 8 High accuracy extraction


In [59]:
print("Forced OCR Configuration for Scanned PDFs:")
print("="*60)

elements = partition_pdf(
    filename="sample_documents\scansmpl.pdf",
    strategy="ocr_only",
    ocr_languages="eng"
)
print(f"Elements from scanned PDF: {len(elements)}")
for elem in elements:
    print(f"{type(elem).__name__}: {elem.text}")

The ocr_languages kwarg will be deprecated in a future version of unstructured. Please use languages instead.
Only one of languages and ocr_languages should be specified. languages is preferred. ocr_languages is marked for deprecation.


Forced OCR Configuration for Scanned PDFs:
Elements from scanned PDF: 13
Title: THE SLEREXE COMPANY LIMITED
Text: SAPORS LANE - BOOLE - DORSET - BH 25 8ER TELEPHONE BOOLE (945 13) 51617 - TELEX 123456
Title: Our Ref. 350/PJC/EAC 18th January, 1972.
Text: Dr. P.N. Cundall, Mining Surveys Ltd., Holroyd Road, Reading,
Title: Berks.
Text: Dear Pete,
NarrativeText: Permit me to introduce you to the facility of facsimile transmission.
NarrativeText: In facsimile a photocell is caused to perform a raster scan over the subject copy. The variations of print density on the document cause the photocell to generate an analogous electrical video signal. This signal is used to modulate a carrier, which is transmitted to a remote destination over a radio or cable communications link.
NarrativeText: At the remote terminal, demodulation reconstructs the video signal, which is used to modulate the density of print produced by a printing device. This device is scanning in a raster scan synchronised with 

### **LangChain Integration**

In [61]:
from langchain_unstructured import UnstructuredLoader
print("Langchain Unstructured Integration")
print("="*60)

loader = UnstructuredLoader(
    file_path="sample_documents/docling_paper.pdf"
)
docs = loader.load()
print(f"Loaded {len(docs)} Langchain documents")

if docs:
    print("First document:")
    print(f"   - Content : {docs[0].page_content[:200]}")
    print(f"   - Metadata: {docs[0].metadata}")

Langchain Unstructured Integration
Loaded 276 Langchain documents
First document:
   - Content : 4 2 0 2 c e D 9
   - Metadata: {'source': 'sample_documents/docling_paper.pdf', 'coordinates': {'points': ((16.34, 216.16000000000008), (16.34, 308.36), (36.34, 308.36), (36.34, 216.16000000000008)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}, 'file_directory': 'sample_documents', 'filename': 'docling_paper.pdf', 'last_modified': '2025-12-30T05:14:11', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/pdf', 'category': 'UncategorizedText', 'element_id': '94656b2552b350a8a9bedcf9fe7ad9dc'}


In [62]:
print("UnstructuredLoader Modes:")
print("="*60)

modes = {
    "single": "Entire document as one Document object",
    "elements": "Each element as separate Document",
    "paged": "Each page as separate Document",
}

for mode, description in modes.items():
    print(f"\n{mode}:")
    print(f"  {description}")

UnstructuredLoader Modes:

single:
  Entire document as one Document object

elements:
  Each element as separate Document

paged:
  Each page as separate Document


In [63]:
loader_elements = UnstructuredLoader(
    file_path="sample_documents/docling_paper.pdf",
    mode="elements"
)
docs_elements = loader_elements.load()
print(f"Documents in 'elements'mode: {len(docs_elements)}")
print()

print("Sample documents:")
for i, doc in enumerate(docs_elements[:5]):
    content = doc.page_content[:80] + "..." if len(doc.page_content) > 80 else doc.page_content
    print(f"  [{i}] {content}")

Documents in 'elements'mode: 276

Sample documents:
  [0] 4 2 0 2 c e D 9
  [1] ] L C . s c [
  [2] 5 v 9 6 8 9 0 . 8 0 4 2 : v i X r a
  [3] Docling Technical Report
  [4] Version 1.0


In [64]:
loader_single = UnstructuredLoader(
    file_path="sample_documents/docling_paper.pdf",
    mode="single",  # Entire document as one
)

docs_single = loader_single.load()

print(f"Documents in 'single' mode: {len(docs_single)}")
print(f"\nDocument length: {len(docs_single[0].page_content)} characters")
print(f"\nFirst 500 characters:")
print(docs_single[11].page_content[:1000])

Documents in 'single' mode: 276

Document length: 15 characters

First 500 characters:
Converting PDF documents back into a machine-processable format has been a major challenge for decades due to their huge variability in formats, weak standardization and printing-optimized characteristic, which discards most structural features and metadata. With the advent of LLMs and popular application patterns such as retrieval-augmented generation (RAG), leveraging the rich content embedded in PDFs has become ever more relevant. In the past decade, several powerful document understanding solutions have emerged on the market, most of which are commercial soft- ware, cloud offerings [3] and most recently, multi-modal vision-language models. As of today, only a handful of open-source tools cover PDF conversion, leaving a significant feature and quality gap to proprietary solutions.


In [65]:
pdf_path = "sample_documents/docling_paper.pdf"
# Load PDF with hi_res strategy
loader_pdf = UnstructuredLoader(
    file_path=pdf_path,
    mode="elements",
    strategy="fast",  # Use fast for this demo
)

docs_pdf = loader_pdf.load()

print(f"PDF documents loaded: {len(docs_pdf)}")
print()

# Show element types in metadata
print("Document metadata examples:")
for doc in docs_pdf[:3]:
    print(f"  Category: {doc.metadata.get('category', 'N/A')}")
    print(f"  Page: {doc.metadata.get('page_number', 'N/A')}")
    print(f"  Content: {doc.page_content[:60]}...")
    print()

PDF documents loaded: 276

Document metadata examples:
  Category: UncategorizedText
  Page: 1
  Content: 4 2 0 2 c e D 9...

  Category: Title
  Page: 1
  Content: ] L C . s c [...

  Category: UncategorizedText
  Page: 1
  Content: 5 v 9 6 8 9 0 . 8 0 4 2 : v i X r a...



In [66]:
import os
files_to_load = [
    "sample_documents/sample.html",
    "sample_documents/sample.md",
]
all_docs = []
for file_path in files_to_load:
    if os.path.exists(file_path):
        loader = UnstructuredLoader(
            file_path=file_path,
            mode="elements",
        )
        docs = loader.load()
        all_docs.extend(docs)
        print(f"Loaded {len(docs)} documents from {file_path}")

print(f"\nTotal documents: {len(all_docs)}")

Loaded 20 documents from sample_documents/sample.html
Loaded 44 documents from sample_documents/sample.md

Total documents: 64


### **Complete RAG Application Example**

In [67]:
!uv pip show langchain

Name: langchain
Version: 1.2.0
Location: F:\sourab\data_parsing\.venv\Lib\site-packages
Requires: langchain-core, langgraph, pydantic
Required-by:


In [ ]:
from langchain_unstructured import UnstructuredLoader
from langchain_chroma import Chroma
from langchain_openai import AzureOpenAI, AzureOpenAIEmbeddings
from lan